In [63]:
import pandas as pd
import glob
import requests
from bs4 import BeautifulSoup
import os

somelist=[]
for week_no in range(1,16):
    
    url = f'https://theaudl.com/league/schedule/week-{week_no}'
    result = requests.get(url)
    
    soup = BeautifulSoup(result.text,'lxml')
    
    matchup_links = [i.find_all('a')[0].attrs['href'] for i in soup.find_all('span',attrs={'class':'audl-schedule-gc-link'})]
    start_times_places = [i.text for i in soup.find_all('span',attrs={'class':'audl-schedule-start-time-text'})]

    scores = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-team-score"})]
    team_names = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-team-name"})]
    score_pairs = [(i,j) for i,j in zip(scores[::2],scores[1::2])]
    team_name_pairs = [(i,j) for i,j in zip(team_names[::2],team_names[1::2])]

    somelist+=[{'Year':2019,'Week':week_no,
                'Matchup Link':mul,'Date Location':stp,
                'Home Team':hm,'Away Team':aw,
                'Home Team Score':hms,'Away Team Score':aws
                } for mul,stp,(aws,hms),(aw,hm) in zip(matchup_links, start_times_places,score_pairs,team_name_pairs)]
         
audl_sched = pd.DataFrame(somelist)
audl_sched.to_csv('../../data/supplemental/schedule.csv')
audl_sched


,Away Team,Away Team Score,Date Location,Home Team,Home Team Score,Matchup Link,Week,Year
0,Raleigh Flyers,17,"Fri, 4/5 7:30 PM CDT Dallas",Dallas Roughnecks,19,/league/game/2019-04-05-RAL-DAL,1,2019
1,Raleigh Flyers,22,"Sat, 4/6 6:00 PM CDT Austin",Austin Sol,18,/league/game/2019-04-06-RAL-AUS,1,2019
2,San Jose Spiders,28,"Sat, 4/6 6:30 PM PDT Seattle",Seattle Cascades,27,/league/game/2019-04-06-SJ-SEA,1,2019
3,Los Angeles Aviators,18,"Sat, 4/6 6:45 PM PDT San Diego",San Diego Growlers,20,/league/game/2019-04-06-LA-SD,1,2019
4,Atlanta Hustle,17,"Sat, 4/6 7:05 PM EDT Tampa Bay",Tampa Bay Cannons,18,/league/game/2019-04-06-ATL-TB,1,2019
5,San Jose Spiders,18,"Fri, 4/12 7:30 PM PDT LA",Los Angeles Aviators,22,/league/game/2019-04-12-SJ-LA,2,2019
6,DC Breeze,18,"Sat, 4/13 6:00 PM EDT New York",New York Empire,21,/league/game/2019-04-13-DC-NY,2,2019
7,Dallas Roughnecks,17,"Sat, 4/13 6:00 PM CDT Austin",Austin Sol,19,/league/game/2019-04-13-DAL-AUS,2,2019
8,San Jose Spiders,24,"Sat, 4/13 6:45 PM PDT San Diego",San Diego Growlers,25,/league/game/2019-04-13-SJ-SD,2,2019
9,Raleigh Flyers,24,"Sat, 4/13 7:00 PM EDT Atlanta",Atlanta Hustle,21,/league/game/2019-04-13-RAL-ATL,2,2019


In [70]:

df_list = []

for year_id, (year,max_pages) in enumerate([[2018,9],[2017,9],[2016,10],
                                            [2015,10],[2014,7],[2013,6],[2012,4]]):

    for page_num in range(0,max_pages):
        url = f'https://theaudl.com/league/game?page={page_num}&year={year_id+1}'
        result = requests.get(url)

        soup = BeautifulSoup(result.text,'lxml')

        matchup_links = [i.find_all('a')[0].attrs['href'] for i in soup.find_all('td',attrs={'class':"audl-schedule-game-center",'colspan':"3"})]
        start_times_places = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-start-time",'colspan':"2"})]

        scores = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-team-score"})]
        team_names = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-team-name"})]
        score_pairs = [(i,j) for i,j in zip(scores[::2],scores[1::2])]
        team_name_pairs = [(i,j) for i,j in zip(team_names[::2],team_names[1::2])]

        somelist=[{'Year':mul.split('/')[-1].split('-')[0],'Week':'?',
                    'Matchup Link':mul,'Date Location':stp,
                    'Home Team':hm,'Away Team':aw,
                    'Home Team Score':hms,'Away Team Score':aws
                    } for mul,stp,(aws,hms),(aw,hm) in zip(matchup_links, start_times_places,score_pairs,team_name_pairs)]

        df_list.append(pd.DataFrame(somelist))
    
pd.concat(df_list)



In [85]:
games = pd.concat(df_list)
games['Date']=games['Matchup Link'].apply(lambda x : '/'.join(x.split('/')[3].split('-')[:3]) )
games['Date'] = pd.to_datetime(games['Date'],format='%Y/%m/%d')
games['Week'] = games['Date'].dt.strftime('%W').astype(int)
games['Week'] = games.apply(lambda x : x.Week - games[games.Year==x.Year].Week.min() + 1,axis=1)
games


,Away Team,Away Team Score,Date Location,Home Team,Home Team Score,Matchup Link,Week,Year,Date
0,Indianapolis AlleyCats,24,"Sat, 3/31 4:00 PM EDT Detroit",Detroit Mechanix,12,/league/game/2018-03-31-IND-DET,1,2018,2018-03-31
1,Tampa Bay Cannons,22,"Sat, 3/31 7:00 PM EDT Raleigh",Raleigh Flyers,27,/league/game/2018-03-31-TB-RAL,1,2018,2018-03-31
2,Minnesota Wind Chill,28,"Sat, 4/7 6:00 PM PDT Seattle",Seattle Cascades,29,/league/game/2018-04-07-MIN-SEA,2,2018,2018-04-07
3,San Diego Growlers,17,"Sat, 4/7 6:00 PM PDT LA",Los Angeles Aviators,30,/league/game/2018-04-07-SD-LA,2,2018,2018-04-07
4,Detroit Mechanix,16,"Sat, 4/7 6:00 PM EDT Pittsburgh",Pittsburgh Thunderbirds,17,/league/game/2018-04-07-DET-PIT,2,2018,2018-04-07
5,Montreal Royal,24,"Sat, 4/7 6:30 PM EDT DC",DC Breeze,22,/league/game/2018-04-07-MTL-DC,2,2018,2018-04-07
6,San Jose Spiders,21,"Sat, 4/7 6:30 PM PDT San Francisco",San Francisco FlameThrowers,20,/league/game/2018-04-07-SJ-SF,2,2018,2018-04-07
7,Atlanta Hustle,17,"Sat, 4/7 7:00 PM EDT Tampa Bay",Tampa Bay Cannons,15,/league/game/2018-04-07-ATL-TB,2,2018,2018-04-07
8,Dallas Roughnecks,17,"Sat, 4/7 7:00 PM EDT Raleigh",Raleigh Flyers,14,/league/game/2018-04-07-DAL-RAL,2,2018,2018-04-07
9,Madison Radicals,20,"Sat, 4/7 7:30 PM EDT Indianapolis",Indianapolis AlleyCats,13,/league/game/2018-04-07-MAD-IND,2,2018,2018-04-07


In [72]:
games.head()

,Away Team,Away Team Score,Date Location,Home Team,Home Team Score,Matchup Link,Week,Year
0,Indianapolis AlleyCats,24,"Sat, 3/31 4:00 PM EDT Detroit",Detroit Mechanix,12,/league/game/2018-03-31-IND-DET,?,2018
1,Tampa Bay Cannons,22,"Sat, 3/31 7:00 PM EDT Raleigh",Raleigh Flyers,27,/league/game/2018-03-31-TB-RAL,?,2018
2,Minnesota Wind Chill,28,"Sat, 4/7 6:00 PM PDT Seattle",Seattle Cascades,29,/league/game/2018-04-07-MIN-SEA,?,2018
3,San Diego Growlers,17,"Sat, 4/7 6:00 PM PDT LA",Los Angeles Aviators,30,/league/game/2018-04-07-SD-LA,?,2018
4,Detroit Mechanix,16,"Sat, 4/7 6:00 PM EDT Pittsburgh",Pittsburgh Thunderbirds,17,/league/game/2018-04-07-DET-PIT,?,2018


In [87]:
# soup.find_all('td',attrs={'class':"audl-schedule-game-center",'colspan':"3"})

In [88]:
# soup.find_all()

# Development of 2019 only

In [48]:
url = f'https://theaudl.com/league/schedule/week-1'
result = requests.get(url)

soup = BeautifulSoup(result.text,'lxml')

In [89]:
# soup.find_all()

# Development

In [35]:
for i in soup.find_all('span',attrs={'class':'audl-schedule-gc-link'}):
    print(i.find_all('a')[0].attrs['href'])
    
[i.find_all('a')[0].attrs['href'] for i in soup.find_all('span',attrs={'class':'audl-schedule-gc-link'})]

/league/game/2019-04-05-RAL-DAL
/league/game/2019-04-06-RAL-AUS
/league/game/2019-04-06-SJ-SEA
/league/game/2019-04-06-LA-SD
/league/game/2019-04-06-ATL-TB


['/league/game/2019-04-05-RAL-DAL',
 '/league/game/2019-04-06-RAL-AUS',
 '/league/game/2019-04-06-SJ-SEA',
 '/league/game/2019-04-06-LA-SD',
 '/league/game/2019-04-06-ATL-TB']

In [15]:
soup.find_all('span',attrs={'class':'audl-schedule-start-time-text'})

[<span class="audl-schedule-start-time-text">Fri, 4/5 7:30 PM CDT Dallas</span>,
 <span class="audl-schedule-start-time-text">Sat, 4/6 6:00 PM CDT Austin</span>,
 <span class="audl-schedule-start-time-text">Sat, 4/6 6:30 PM PDT Seattle</span>,
 <span class="audl-schedule-start-time-text">Sat, 4/6 6:45 PM PDT San Diego</span>,
 <span class="audl-schedule-start-time-text">Sat, 4/6 7:05 PM EDT Tampa Bay</span>]

In [12]:
soup.find_all('td',attrs={'class':"audl-schedule-team-score"})

[<td class="audl-schedule-team-score">17</td>,
 <td class="audl-schedule-team-score">19</td>,
 <td class="audl-schedule-team-score">22</td>,
 <td class="audl-schedule-team-score">18</td>,
 <td class="audl-schedule-team-score">28</td>,
 <td class="audl-schedule-team-score">27</td>,
 <td class="audl-schedule-team-score">18</td>,
 <td class="audl-schedule-team-score">20</td>,
 <td class="audl-schedule-team-score">17</td>,
 <td class="audl-schedule-team-score">18</td>]

In [16]:
soup.find_all('td',attrs={'class':"audl-schedule-team-name"})

[<td class="audl-schedule-team-name"><a href="/flyers">Raleigh Flyers</a></td>,
 <td class="audl-schedule-team-name"><a href="/roughnecks">Dallas Roughnecks</a></td>,
 <td class="audl-schedule-team-name"><a href="/flyers">Raleigh Flyers</a></td>,
 <td class="audl-schedule-team-name"><a href="/sol">Austin Sol</a></td>,
 <td class="audl-schedule-team-name"><a href="/spiders">San Jose Spiders</a></td>,
 <td class="audl-schedule-team-name"><a href="/cascades">Seattle Cascades</a></td>,
 <td class="audl-schedule-team-name"><a href="/aviators">Los Angeles Aviators</a></td>,
 <td class="audl-schedule-team-name"><a href="/growlers">San Diego Growlers</a></td>,
 <td class="audl-schedule-team-name"><a href="/hustle">Atlanta Hustle</a></td>,
 <td class="audl-schedule-team-name"><a href="/cannons">Tampa Bay Cannons</a></td>]

In [90]:
# soup.find_all('div',attrs={'class':"view-content"})[0]#all weeks/links

# for i in soup.find_all('div',attrs={'class':"view-content"})[1].find_all('td'):#games
#     print(i)
#     print()


# Test whole script

In [86]:
import pandas as pd
import glob
import requests
from bs4 import BeautifulSoup
import os

## Previous Years
df_list = []

for year_id, (year,max_pages) in enumerate([[2018,9],[2017,9],[2016,10],
                                            [2015,10],[2014,7],[2013,6],[2012,4]]):

    for page_num in range(0,max_pages):
        url = f'https://theaudl.com/league/game?page={page_num}&year={year_id+1}'
        result = requests.get(url)

        soup = BeautifulSoup(result.text,'lxml')

        matchup_links = [i.find_all('a')[0].attrs['href'] for i in soup.find_all('td',attrs={'class':"audl-schedule-game-center",'colspan':"3"})]
        start_times_places = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-start-time",'colspan':"2"})]

        scores = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-team-score"})]
        team_names = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-team-name"})]
        score_pairs = [(i,j) for i,j in zip(scores[::2],scores[1::2])]
        team_name_pairs = [(i,j) for i,j in zip(team_names[::2],team_names[1::2])]

        somelist=[{'Year':mul.split('/')[-1].split('-')[0],'Week':'?',
                    'Matchup Link':mul,'Date Location':stp,
                    'Home Team':hm,'Away Team':aw,
                    'Home Team Score':hms,'Away Team Score':aws
                    } for mul,stp,(aws,hms),(aw,hm) in zip(matchup_links, start_times_places,score_pairs,team_name_pairs)]

        df_list.append(pd.DataFrame(somelist))
    

## 2019 / current year
somelist=[]
for week_no in range(1,16):
    
    url = f'https://theaudl.com/league/schedule/week-{week_no}'
    result = requests.get(url)
    
    soup = BeautifulSoup(result.text,'lxml')
    
    matchup_links = [i.find_all('a')[0].attrs['href'] for i in soup.find_all('span',attrs={'class':'audl-schedule-gc-link'})]
    start_times_places = [i.text for i in soup.find_all('span',attrs={'class':'audl-schedule-start-time-text'})]

    scores = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-team-score"})]
    team_names = [i.text for i in soup.find_all('td',attrs={'class':"audl-schedule-team-name"})]
    score_pairs = [(i,j) for i,j in zip(scores[::2],scores[1::2])]
    team_name_pairs = [(i,j) for i,j in zip(team_names[::2],team_names[1::2])]

    somelist+=[{'Year':2019,'Week':week_no,
                'Matchup Link':mul,'Date Location':stp,
                'Home Team':hm,'Away Team':aw,
                'Home Team Score':hms,'Away Team Score':aws
                } for mul,stp,(aws,hms),(aw,hm) in zip(matchup_links, start_times_places,score_pairs,team_name_pairs)]
         
games = pd.concat([pd.DataFrame(somelist)]+df_list)

# Fill in Week Number
games['Date'] = games['Matchup Link'].apply(lambda x : '/'.join(x.split('/')[3].split('-')[:3]) )
games['Date'] = pd.to_datetime(games['Date'],format='%Y/%m/%d')
games['Week'] = games['Date'].dt.strftime('%W').astype(int)
games['Week'] = games.apply(lambda x : x.Week - games[games.Year==x.Year].Week.min() + 1,axis=1)

games#.to_csv('../../data/supplemental/games.csv')



,Away Team,Away Team Score,Date Location,Home Team,Home Team Score,Matchup Link,Week,Year,Date
0,Raleigh Flyers,17,"Fri, 4/5 7:30 PM CDT Dallas",Dallas Roughnecks,19,/league/game/2019-04-05-RAL-DAL,1,2019,2019-04-05
1,Raleigh Flyers,22,"Sat, 4/6 6:00 PM CDT Austin",Austin Sol,18,/league/game/2019-04-06-RAL-AUS,1,2019,2019-04-06
2,San Jose Spiders,28,"Sat, 4/6 6:30 PM PDT Seattle",Seattle Cascades,27,/league/game/2019-04-06-SJ-SEA,1,2019,2019-04-06
3,Los Angeles Aviators,18,"Sat, 4/6 6:45 PM PDT San Diego",San Diego Growlers,20,/league/game/2019-04-06-LA-SD,1,2019,2019-04-06
4,Atlanta Hustle,17,"Sat, 4/6 7:05 PM EDT Tampa Bay",Tampa Bay Cannons,18,/league/game/2019-04-06-ATL-TB,1,2019,2019-04-06
5,San Jose Spiders,18,"Fri, 4/12 7:30 PM PDT LA",Los Angeles Aviators,22,/league/game/2019-04-12-SJ-LA,2,2019,2019-04-12
6,DC Breeze,18,"Sat, 4/13 6:00 PM EDT New York",New York Empire,21,/league/game/2019-04-13-DC-NY,2,2019,2019-04-13
7,Dallas Roughnecks,17,"Sat, 4/13 6:00 PM CDT Austin",Austin Sol,19,/league/game/2019-04-13-DAL-AUS,2,2019,2019-04-13
8,San Jose Spiders,24,"Sat, 4/13 6:45 PM PDT San Diego",San Diego Growlers,25,/league/game/2019-04-13-SJ-SD,2,2019,2019-04-13
9,Raleigh Flyers,24,"Sat, 4/13 7:00 PM EDT Atlanta",Atlanta Hustle,21,/league/game/2019-04-13-RAL-ATL,2,2019,2019-04-13
